# Import packages for the code

In [ ]:
import skimage as sk
import numpy as np
import os
import napari
import pandas as pd
from glob import glob
from tqdm import tqdm

# Functions that carry out all steps of the script
Get the object IDs and volumes, to scale, of each lesion. In Scikit-image, the area measurement is also the volume measurement for 3D objects. Before getting measurements, any objects touching the border of the image are removed (these are incomplete objects and will not have representative measurements).

In [ ]:
def get_measurements(mask,props, voxels):
    filtered_masks = sk.segmentation.clear_border(mask)
    df = sk.measure.regionprops_table(filtered_masks,properties=props, spacing=voxels)
    df = pd.DataFrame.from_dict(df)
    return filtered_masks, df

Calculate the surface area of each object, this is the most computationally intense step. Add the surface area to the existing data frame created from the previous function.

In [ ]:
def get_3D_surfacearea_and_sphericity(filtered_masks,df,voxels):
    surf_area_list = []
    sphericity_list = []
    vol = np.asarray(df['area']).astype(np.float64)
    c = 0
    for i in np.asarray(df['label']).astype(int):
        obj = filtered_masks == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0,spacing=(voxels[0],voxels[1],voxels[2]))
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        surf_area_list.append(surf_area)
        sphericity = (np.pi**(1/3)*((6*vol[c])**(2/3)))/surf_area
        sphericity_list.append(sphericity)
        c += 1
    surface_areas = pd.Series(surf_area_list,name='Surface_Area (um^2)')
    sphericities = pd.Series(sphericity_list,name='Sphericity')
    merged_df = pd.concat([df,surface_areas,sphericities], axis=1)
    return merged_df
    

Create a folder in a specified location to save the final label images and data frames.

In [ ]:
def save(save_path, img_name, filtered_masks, merged_df):
    # make directories if they do not exist
    try:
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        dataframe_path = os.path.join(save_path,'measurements')
    sk.io.imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'.tif'),filtered_masks,check_contrast=False)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

# Read in the images and then use the functions in a loop to get and save measurements

In [ ]:
files = sorted(glob('path/to/images/*.tif')) #gets a list of all files that end with a .tif extension
labels = list(map(sk.io.imread,files)) #read in all images as an array (required for doing anything with the image data)

In [ ]:
viewer = napari.view_labels(labels[10]) #Optional, use to visually check that the label images look correct

Specify the properties to measure with scikit-image, the size of the pixel in 3d (voxel), and the save location. Then run the functions defined above in a loop over all the images. tqdm is a python package that shows a progress bar when a loop is running. It's not needed but very helpful to check the progress.

In [ ]:
props = ('label','area')
voxels = [3.0,0.208,0.208]
save_path = '/path/to/save/location/'

for i in tqdm(range(len(labels))):
    img_name = os.path.basename(files[i])
    mask_img, df = get_measurements(mask=labels[i],props=props,voxels=voxels)
    merged_df = get_3D_surfacearea_and_sphericity(filtered_masks=mask_img,df=df,voxels=voxels)
    save(save_path=save_path, img_name=img_name, filtered_masks=mask_img, merged_df=merged_df)